In [1]:
import os
import os.path
import sys
import pandas as pd
import numpy as np
from importlib import reload

In [2]:
LOCAL_PATH = 'data'
RAW_TRACE = 'Skype_HongKong.pcapng'
TCP_TRACE_FEATURE_FILE = 'Skype_HongKong_tcp_pkt.csv'
UDP_TRACE_FEATURE_FILE = 'Skype_HongKong_udp_pkt.csv'
TCP_FLOW_FEATURE_FILE = 'Skype_HongKong_tcp_flow.csv'
UDP_FLOW_FEATURE_FILE = 'Skype_HongKong_udp_flow.csv'
# BUCKET_NAME = 'edu.nyu.hsn.ddos-data' # replace with your bucket name
# KEY = 'CAP_NIC1_00931_20130727230801.dms' # replace with your object key

In [3]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [4]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
%time packet_feature.tcp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) ,os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))

Packet feature file already exists.
CPU times: user 111 µs, sys: 58 µs, total: 169 µs
Wall time: 148 µs
Packet feature file already exists.
CPU times: user 302 µs, sys: 121 µs, total: 423 µs
Wall time: 334 µs


In [5]:
def to_csv(trace_df, extract_func, file):
    extract_func(trace_df, 1.0, upsampled=True).to_csv(file, index=False)

In [8]:
from python import flow_feature
reload(flow_feature)

tcp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))
udp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
for trace_df in [tcp_trace_df, udp_trace_df]:
    if tcp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.tcp_generate, os.path.join(LOCAL_PATH, TCP_FLOW_FEATURE_FILE))
    elif udp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.udp_generate, os.path.join(LOCAL_PATH, UDP_FLOW_FEATURE_FILE))
    else:
        raise



TCP 1.0 samp rate with upsampling:   0%|          | 0/726 [00:00<?, ?it/s]

TCP 1.0 samp rate with upsampling:   1%|          | 8/726 [00:00<00:09, 77.07it/s]

TCP 1.0 samp rate with upsampling:   2%|▏         | 17/726 [00:00<00:08, 79.36it/s]

TCP 1.0 samp rate with upsampling:   4%|▎         | 27/726 [00:00<00:08, 83.42it/s]

TCP 1.0 samp rate with upsampling:   5%|▌         | 37/726 [00:00<00:08, 85.30it/s]

TCP 1.0 samp rate with upsampling:   6%|▋         | 47/726 [00:00<00:07, 87.57it/s]

TCP 1.0 samp rate with upsampling:   8%|▊         | 57/726 [00:00<00:07, 89.86it/s]

TCP 1.0 samp rate with upsampling:   9%|▉         | 67/726 [00:00<00:07, 91.63it/s]

TCP 1.0 samp rate with upsampling:  10%|█         | 76/726 [00:00<00:07, 88.52it/s]

TCP 1.0 samp rate with upsampling:  12%|█▏        | 85/726 [00:00<00:07, 85.72it/s]

TCP 1.0 samp rate with upsampling:  13%|█▎        | 95/726 [00:01<00:07, 87.17it/s]

TCP 1.0 samp rate with upsampling:  14%|█▍        | 105/726 [00:01<00:06,

['90.217.83.107,192.168.1.163:6769', '90.217.83.107:28331', '192.168.1.163:6769']
['90.217.83.107,192.168.1.163:6769', '90.217.83.107:28331', '192.168.1.163:6769']


AssertionError: 



UDP 1.0 samp rate with upsampling:   5%|▌         | 11/216 [00:14<00:03, 53.26it/s]